In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
## data loading
data = pd.read_csv("/home/gray/Downloads/integrated.csv")

In [37]:
data.head(3)
# data.info()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,budget,genres,homepage,id,keywords,original_language,original_title,...,actor1_hashtags,actor2_hashtags,actor3_hashtags,director_hashtags,movie_hashtags,IMDB_ID,genres.1,production_companies.1,budgets,gross_usa
0,0,0,0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,...,16431,141212,82306,107952,1744076,tt0499549,"[{'name': 'Action'}, {'name': 'Adventure'}, {'...","[{'name': 'Twentieth Century Fox'}, {'name': '...",237000000,749700000
1,1,1,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,...,1946081,239740,164826,5386,1600,tt0449088,"[{'name': 'Action'}, {'name': 'Adventure'}, {'...","[{'name': 'Walt Disney Pictures'}, {'name': 'J...",300000000,309404152
2,2,2,2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,...,179939,45185,28352,18433,386542,tt2379713,"[{'name': 'Action'}, {'name': 'Adventure'}, {'...","[{'name': 'B24'}, {'name': 'Columbia Pictures ...",245000000,208777731


In [66]:
def filter_operation_1(data, threshold_gross_usa, threshold_budges, threshold_ratio):
    '''
    1st operation:
    Filter the data to eleminate the string and the data which are lower than thresholds.
    '''
    filter_data_1 = data
    filter_data_1['budgets'] = pd.to_numeric(filter_data_1['budgets'], errors='coerce')
    filter_data_1['gross_usa'] = pd.to_numeric(filter_data_1['gross_usa'], errors='coerce')
    print(len(filter_data_1))
    filter_data_1 = filter_data_1.dropna(subset=['budgets'])
    filter_data_1 = filter_data_1.dropna(subset=['gross_usa'])
    print(len(filter_data_1))
    filter_data_1 = filter_data_1[(filter_data_1['budgets'] >= threshold_budges) & 
                                  (filter_data_1['gross_usa'] >= threshold_gross_usa) & 
                                  (filter_data_1['gross_usa'] / filter_data_1['budgets'] <= threshold_ratio[1]) & 
                                  (filter_data_1['gross_usa'] / filter_data_1['budgets'] >= threshold_ratio[0])]
    print(len(filter_data_1))
    
    return filter_data_1

threshold_gross_usa = 1000
threshold_budges = 1000
threshold_ratio = [1e-3, 1e6]
filter_data_1 = filter_operation_1(data.copy(), threshold_gross_usa, threshold_budges, threshold_ratio)

4803
3743
3602


In [64]:
filter_data_1.head(3)
filter_data_1[filter_data_1['budgets'] / filter_data_1['budgets'] < 0]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,budget,genres,homepage,id,keywords,original_language,original_title,...,actor1_hashtags,actor2_hashtags,actor3_hashtags,director_hashtags,movie_hashtags,IMDB_ID,genres.1,production_companies.1,budgets,gross_usa


In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 37 columns):
Unnamed: 0                4803 non-null int64
Unnamed: 0.1              4803 non-null int64
Unnamed: 0.1.1            4803 non-null int64
budget                    4803 non-null int64
genres                    4803 non-null object
homepage                  1712 non-null object
id                        4803 non-null int64
keywords                  4803 non-null object
original_language         4803 non-null object
original_title            4803 non-null object
overview                  4800 non-null object
popularity                4803 non-null float64
production_companies      4803 non-null object
production_countries      4803 non-null object
release_date              4802 non-null object
revenue                   4803 non-null int64
runtime                   4801 non-null float64
spoken_languages          4803 non-null object
status                    4803 non-null object
tagl

In [67]:
def statistics_mean_std(data, item_list=[1, 2]):
    '''
    the position of the following statistics are [0, 1, 2, 3, 4, 5, 6, 7]
    count, mean, std, min, 25%, 50%, 75%, max 
    '''
    output = []
    record = data.describe()
    for each_item in item_list:
        output.append(record[each_item])
    return output

avg, std = statistics_mean_std(filter_data_1['budgets'])
# print(statistics_mean_std(data['budgets']))
print(avg + 3 * std)

avg, std = statistics_mean_std(filter_data_1['gross_usa'])
print(avg + 3 * std)

166225497.9559851
296751465.28885233


In [68]:
def filter_operation_2(data, threshold, labels=['budgets', 'gross_usa']):
    '''
    filter the data with average + threshold * std
    '''
    output_data = data
    for each_label in labels:
        avg, std = statistics_mean_std(data[each_label])
        boundary = avg + std * threshold
        output_data = output_data[output_data[each_label] <= boundary]
    
    return output_data

filter_data_2 = filter_operation_2(filter_data_1.copy(), 3)


In [73]:
## Get the statistics of AB - AF (actor1_hashtags, actor2_hashtags, actor3_hashtags, director_hashtags, movie_hashtags)
candidate_labels = ['actor1_hashtags', 'actor2_hashtags', 'actor3_hashtags', 'director_hashtags', 'movie_hashtags']

output = {}
for each_label in candidate_labels:
    avg, std = statistics_mean_std(filter_data_2[each_label])
#     output[each_label] = [avg, std]
    output[each_label] = avg + 3 * std

print(output)

avg, std = statistics_mean_std(filter_data_2['budgets'])
# print(statistics_mean_std(data['budgets']))
print(avg + 3 * std)

avg, std = statistics_mean_std(filter_data_2['gross_usa'])
print(avg + 3 * std)

{'director_hashtags': 443812.27163350454, 'movie_hashtags': 18026577.874490555, 'actor1_hashtags': 3774622.803967894, 'actor3_hashtags': 6355893.462638536, 'actor2_hashtags': 2934441.3803638546}
134831243.12654257
199924911.7174828


In [74]:
filter_data_3 = filter_operation_2(filter_data_2.copy(), 3, labels=candidate_labels)

output = {}
for each_label in candidate_labels:
    avg, std = statistics_mean_std(filter_data_3[each_label])
#     output[each_label] = [avg, std]
    output[each_label] = avg + 3 * std

print(output)

avg, std = statistics_mean_std(filter_data_3['budgets'])
# print(statistics_mean_std(data['budgets']))
print(avg + 3 * std)

avg, std = statistics_mean_std(filter_data_3['gross_usa'])
print(avg + 3 * std)

{'director_hashtags': 184364.03222806207, 'movie_hashtags': 4945671.31175635, 'actor1_hashtags': 1199089.5904073408, 'actor3_hashtags': 808170.6934094004, 'actor2_hashtags': 794356.0146188609}
134633568.58594722
198835033.4998501


In [75]:
def correlation(data, basic_label, candidate_labels):
    '''
    Calculate the .corr() between basic_label with each one of candidate_labels
    '''
    output = {}
    for each_label in candidate_labels:
        output[basic_label + '-' + each_label] = data[basic_label].corr(data[each_label])
    return output

basic_label = 'gross_usa'

## To AJ -> budgets
print(correlation(filter_data_3, basic_label, ['budgets']))

## To AB - AF: candidate_labels
print(list(correlation(filter_data_3, basic_label, candidate_labels).values()))

## To AB + AC + AD:
candidate_label_AB_AC_AD = ['actor1_hashtags', 'actor2_hashtags', 'actor3_hashtags']
sum_AB_AC_AD = filter_data_3[candidate_label_AB_AC_AD].sum(axis=1) # column summation
print(filter_data_3[basic_label].corr(sum_AB_AC_AD))

## To max(AB, AC, AD):
max_AB_AC_AD = filter_data_3[candidate_label_AB_AC_AD].max(axis=1) # column summation
print(filter_data_3[basic_label].corr(max_AB_AC_AD))

## To max(AB, AC, AD, AE):
candidate_label_AB_AC_AD_AE = ['actor1_hashtags', 'actor2_hashtags', 'actor3_hashtags', 'director_hashtags']
max_AB_AC_AD_AE = filter_data_3[candidate_label_AB_AC_AD_AE].max(axis=1) # column summation
print(filter_data_3[basic_label].corr(max_AB_AC_AD_AE))



{'gross_usa-budgets': 0.5915412418749759}
[0.09714685508884563, 0.12008133971684323, 0.03968436253391814, 0.08870001671494913, 0.19398861693537006]
0.229316480984445
0.17891511148206696
0.17880127654478603
